In [7]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import openpyxl
import os

def scrape_season(start_year, end_year):

    season_year = {}

    for year in range(start_year,end_year):

        #print('Get the colleges from {year} year'.format(year = year))

        url = 'https://www.sports-reference.com/cbb/seasons/{year}.html'.format(year = str(year))

        # collect HTML data
        html = urlopen(url)
                
        # create beautiful soup object from HTML
        soup = BeautifulSoup(html, features="html")

        # get table
        table = soup.find('table', id='conference-summary')

        # Collecting Data
        conference_data = []
        for row in table.tbody.find_all('tr'):    
            conference_data.append(row)

        rows_data = [[td.getText().replace('-', ' ') for td in conference_data[i].findAll('td')]
                            for i in range(len(conference_data))]

        conference_list =[]

        for conference in rows_data[0:]:
            conference_list.append(conference[0])

        hyperlinks_dict = {}
        for link in table.tbody.find_all('a'):
            if link.get_text() in conference_list:
                hyperlinks_dict[link.get_text()] = link.get('href')

        
        season_dict = {}
        season_dict['season'] = hyperlinks_dict
        season_year[str(year)] = season_dict

    return season_year







In [156]:
start_year = 1987
end_year = 2023

season_links = scrape_season(start_year, end_year)

In [4]:
def find_colleges_links(season_links):

    for year in season_links:

        conference_dict = {}
        for conference in season_links[year]['season']:

            url = season_links[year]['season'][conference]

            # collect HTML data
            root_link = 'https://www.sports-reference.com'
            url = root_link + url
            html = urlopen(url)
                    
            # create beautiful soup object from HTML
            soup = BeautifulSoup(html, features="html")

            # get table
            table = soup.find('table', id='standings')

            # Collecting Data
            college_data = []
            for row in table.tbody.find_all('tr'):    
                college_data.append(row)

            rows_data = [[td.getText().replace('-', ' ') for td in college_data[i].findAll('td')]
                                for i in range(len(college_data))]

            college_list =[]

            for college in rows_data[0:]:
                if(len(college) > 0):
                    college_list.append(college[0])

            hyperlinks_dict = {}
            for link in table.tbody.find_all('a'):
                if link.get_text() in college_list:
                    hyperlinks_dict[link.get_text()] = link.get('href')

            
            conference_dict[conference] = hyperlinks_dict

        season_links[year]['college'] = conference_dict
  

    return season_links

            


In [166]:
global_dict = find_colleges_links(season_links)

In [167]:
global_dict['2021']['college']['Big Ten Conference'].keys()

dict_keys(['Michigan', 'Illinois', 'Iowa', 'Purdue', 'Ohio State', 'Wisconsin', 'Rutgers', 'Maryland', 'Michigan State', 'Indiana', 'Penn State', 'Northwestern', 'Minnesota', 'Nebraska'])

In [5]:
def get_players_links(season_links):

    players_links = []
    for year in season_links:

        print(year)

        
        for conferance in season_links[year]['college']:
            
            
            for college in season_links[year]['college'][conferance]:

                url = season_links[year]['college'][conferance][college]

                # collect HTML data
                root_link = 'https://www.sports-reference.com'
                url = root_link + url
                html = urlopen(url)

                
                        
                # create beautiful soup object from HTML
                soup = BeautifulSoup(html, features="html")

                # get table
                table = soup.find('table', id='roster')

                # Collecting Data
                if table is not None:
                    for row in table.tbody.find_all('tr'):    
                        # Find all data for each column
                        link = row.find_all('a')[0]
                        
                        if link.get('href') not in players_links:
                            players_links.append(link.get('href'))


                else:
                    
                    print('The following link does not have the desirable table ' + url)

                

    return players_links

    

In [169]:
players_links = get_players_links(global_dict)

1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
The following link does not have the desirable table https://www.sports-reference.com/cbb/schools/brown/2021.html
The following link does not have the desirable table https://www.sports-reference.com/cbb/schools/columbia/2021.html
The following link does not have the desirable table https://www.sports-reference.com/cbb/schools/cornell/2021.html
The following link does not have the desirable table https://www.sports-reference.com/cbb/schools/dartmouth/2021.html
The following link does not have the desirable table https://www.sports-reference.com/cbb/schools/harvard/2021.html
The following link does not have the desirable table https://www.sports-reference.com/cbb/schools/pennsylvania/2021.html
The following link does not have the desirable table https://www.sports-reference.com/cbb/schools/princeton/2021.html
The f

In [171]:
players_links[0:5]
len(players_links)

54283

In [6]:
def load_player_links(excel_path):

    df = pd.read_excel(excel_path)

    links = df['link']

    return links

In [9]:
folder = 'data'
excel = 'links_players_1987-2022.xlsx'
file_path = os.path.join(folder, excel)
players_links = load_player_links(file_path)

In [10]:
def players_links_excel(players_links):

    id_list = []

    for id in range(len(players_links)):
        id_list.append(id)

    links_players_df = pd.DataFrame()
    links_players_df['id'] = id_list
    links_players_df['link'] = players_links

    links_players_df.to_excel("data/links_players_1987-2022.xlsx")



In [478]:
players_links_excel(players_links)

In [242]:
players_links[20000]

'/cbb/players/roger-mason-1.html'

In [ ]:
25 until 1991
30 until 2011
35 until now


In [252]:
a = []
b = []
c = []
d = []
e = []
a, b, c, d, e = player_data_college_reference(players_links[50000], 1, a, b, c, d, e)

In [255]:
def advanced_table_formatter(data_list, standard_length):

    data_formatted = []
    for index, data in enumerate(data_list):

        data_formatted.append(data)

        # Gert data to format
        if index == 11:
            data_formatted.append('-')
            
        if index == len(data_list) - 1:
            
            while(len(data_formatted)) < standard_length:
                data_formatted.append('-')

    return data_formatted

In [256]:
def get_table_data(soup, table_id, general_data, output_list, pointer_3_year):

    # get table player_per game
    table = soup.find('table', id=table_id)

    # Collecting Data
    if table is not None:
        data = []
        for row in table.tbody.find_all('tr'):    
            data.append(row)
        
        for i in range(len(data)):
            get_season_year = int(data[i].find_all('a')[0].get_text().split('-')[0][0:2] + data[i].find_all('a')[0].get_text().split('-')[1])
            if get_season_year >= pointer_3_year:
                general_data_season = general_data.copy()
                general_data_season.append(get_season_year)
                stats_data = [td.getText() for td in data[i].findAll('td')]
                output_data = general_data_season + stats_data

                if table_id == 'players_advanced':
                    standard_length = 35
                    if len(output_data) < standard_length:
                        output_data = advanced_table_formatter(output_data, standard_length)
                        
                output_list.append(output_data)

                

    return output_list

In [257]:
def player_data_college_reference(player_url, id, per_game_list, total_list, per_40_min_list, per_100_poss_list, advanced_list):

    global_url = 'https://www.sports-reference.com'
    url = global_url + player_url

    html = urlopen(url)

    pointer_3_year = 1987
                        
    # create beautiful soup object from HTML
    soup = BeautifulSoup(html, features="html")

    #Get player info
    info = soup.find(id = 'info')
    player_name = info.find_all('span')[0].get_text()
    info_draft = [data.getText() for data in info.find_all('strong')]

    if len([data.get_text().replace('\n', '').replace(' ', '') for index, data in enumerate(info.find_all('p')[0])]) > 2:
        info_general = [data.get_text().replace('\n', '').replace(' ', '') for index, data in enumerate(info.find_all('p')[0])]
    else:
        info_general = [data.get_text().replace('\n', '').replace(' ', '') for index, data in enumerate(info.find_all('p')[1])]

    position = 'unknown'
    if 'Position:' in info_general:
        position_index = info_general.index('Position:') + 1
        position = info_general[position_index]
    elif len(info_general) < 2:
        print(id, url)
        

    height_cm = 'unknown'
    height_feet = 'unknown' 
    if 'cm' in ' '.join(info_general):
        for index in range(len(info_general)):
            if 'cm' in info_general[index]:
                heigth_index =index
                height_cm = info_general[heigth_index].split('(')[1].split(',')[0].replace('cm', '')[0:3]
                height_feet = info_general[heigth_index].split()[0].replace(',', '')
                break  
    elif len(info_general) < 2:
        print(id, url)
    
   
    
    if 'Draft:' in info_draft:
        draft = 1
    else:
        draft = 0

    general_data = [id, player_name, position, height_cm, height_feet, draft]


    # get table player_per game
    get_table_data(soup, 'players_per_game',general_data, per_game_list, pointer_3_year)

    # get table player totals
    get_table_data(soup, 'players_totals',general_data, total_list, pointer_3_year)

    # get table 40 mins
    get_table_data(soup, 'players_per_min',general_data, per_40_min_list, pointer_3_year)

    # get per 100 poss
    get_table_data(soup, 'players_per_poss',general_data, per_100_poss_list, pointer_3_year)

    # Advanced
    get_table_data(soup, 'players_advanced',general_data, advanced_list, pointer_3_year)

    return per_game_list, total_list, per_40_min_list, per_100_poss_list, advanced_list        




In [270]:
def get_excel_raw_college(players_links):

    per_game_headers = ['id', 'name', 'position', 'heigth_cm', 'heigth_feet', 'draft',
                        'season', 'school', 'conf', 'games', 'games_started', 'mp', 'fg', 'fga', 'fg%',
                        '2p', '2pa', '2p%', '3p', '3pa', '3p%', 'ft', 'fta', 'ft%', 'orb', 'drb', 'tbr', 
                        'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'unknown', 'strength_of_schedule']

    total_headers = ['id', 'name', 'position', 'heigth_cm', 'heigth_feet', 'draft',
                        'season', 'school', 'conf', 'games', 'games_started', 'mp', 'fg', 'fga', 'fg%',
                        '2p', '2pa', '2p%', '3p', '3pa', '3p%', 'ft', 'fta', 'ft%', 'orb', 'drb', 'tbr', 
                        'ast', 'stl', 'blk', 'tov', 'pf', 'pts']

    per_40_headers = ['id', 'name', 'position', 'heigth_cm', 'heigth_feet', 'draft',
                        'season', 'school', 'conf', 'games', 'games_started', 'mp', 'fg', 'fga', 'fg%',
                        '2p', '2pa', '2p%', '3p', '3pa', '3p%', 'ft', 'fta', 'ft%', 'tbr', 
                        'ast', 'stl', 'blk', 'tov', 'pf', 'pts']

    per_100_headers = ['id', 'name', 'position', 'heigth_cm', 'heigth_feet', 'draft',
                        'season', 'school', 'conf', 'games', 'games_started', 'mp', 'fg', 'fga', 'fg%',
                        '2p', '2pa', '2p%', '3p', '3pa', '3p%', 'ft', 'fta', 'ft%', 'tbr', 
                        'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'unknown', 'offensive_rating', 'defensive_rating']

    advanced_headers = ['id', 'name', 'position', 'heigth_cm', 'heigth_feet', 'draft', 'season', 'school', 'conf', 'games',
                        'games_started', 'mp', 'per', 'ts%', 'efg%', '3par', 'ftr', 'pprod', 'orb%', 'drb%', 'trb%', 'ast%',
                        'stl%', 'blk%', 'tov%', 'usg%', 'unknown', 'ows', 'dws', 'ws', 'ws/40', 'unknown', 'obpm', 'dbpm', 'bpm']


    per_game_list = []
    total_list = []
    per_40_min_list = []
    per_100_poss_list = []
    advanced_list = []

    print('Start scrapping data')

    for id, link in enumerate(players_links):
        try:

            player_data_college_reference(link, id, per_game_list, total_list, per_40_min_list, per_100_poss_list, advanced_list)
        except:
            print(id, link)

    print('SCRAPING DATA DONE')

    print('PER GAME TO DF START')
    per_game_df =pd.DataFrame(per_game_list, columns = per_game_headers)
    print('PER GAME TO DF START')

    print('TOTAL TO DF START')
    total_df =pd.DataFrame(total_list, columns = total_headers)
    print('TOTAL TO DF START')

    print('40 MIN TO DF START')
    per_40_min_df =pd.DataFrame(per_40_min_list, columns = per_40_headers)
    print('40 MIN TO DF START')

    print('PER 100 POSS TO DF START')
    per_100_poss_df =pd.DataFrame(per_100_poss_list, columns = per_100_headers)
    print('PER 100 POSS TO DF START')

    print('ADVANCED TO DF START')
    advanced_df =pd.DataFrame(advanced_list, columns = advanced_headers)
    print('ADVANCED TO DF START')
    

    return per_game_df, total_df, per_40_min_df, per_100_poss_df, advanced_df

In [271]:
per_game_df, total_df, per_40_min_df, per_100_poss_df, advanced_df = get_excel_raw_college(players_links[0:1000])

35
Start scrapping data


In [202]:
per_game_df.head()

,id,name,position,heigth_cm,heigth_feet,draft,season,school,conf,games,...,drb,tbr,ast,stl,blk,tov,pf,pts,unknown,strength_of_schedule
0,0,Kenny Smith,Guard,190,6-3,1,1987,UNC,ACC,34,...,,2.2,6.1,1.5,0.1,3.1,1.4,16.9,,8.71
1,1,J.R. Reid Jr,Forward,206,6-9,1,1987,UNC,ACC,36,...,,7.4,1.8,1.3,0.8,2.6,3.1,14.7,,8.71
2,1,J.R. Reid Jr,Forward,206,6-9,1,1988,UNC,ACC,33,...,,8.9,1.7,1.2,1.2,3.8,3.4,18.0,,10.72
3,1,J.R. Reid Jr,Forward,206,6-9,1,1989,UNC,ACC,27,...,,6.3,1.3,0.9,0.8,2.6,3.0,15.9,,12.41
4,2,Joe Wolf,Forward,211,6-11,1,1987,UNC,ACC,34,...,,7.1,2.9,1.3,0.3,1.7,2.5,15.2,,8.71


In [203]:
total_df.head()

,id,name,position,heigth_cm,heigth_feet,draft,season,school,conf,games,...,ft%,orb,drb,tbr,ast,stl,blk,tov,pf,pts
0,0,Kenny Smith,Guard,190,6-3,1,1987,UNC,ACC,34,...,.807,,,76,209,51,3,104,47,574
1,1,J.R. Reid Jr,Forward,206,6-9,1,1987,UNC,ACC,36,...,.653,,,268,66,47,27,94,110,528
2,1,J.R. Reid Jr,Forward,206,6-9,1,1988,UNC,ACC,33,...,.680,,,293,57,39,38,127,113,595
3,1,J.R. Reid Jr,Forward,206,6-9,1,1989,UNC,ACC,27,...,.669,,,170,36,23,21,71,81,429
4,2,Joe Wolf,Forward,211,6-11,1,1987,UNC,ACC,34,...,.793,,,240,99,45,11,57,85,516


In [ ]:
per_40_min_df.head()

In [ ]:
per_100_poss_df.head()

In [ ]:
advanced_df.head()